In [3]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
import threading
import gensim
import logging
import random
from gensim.models import FastText
from gensim.similarities.index import AnnoyIndexer
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [4]:
import glob
import os
import six
import threading
from gensim.models import KeyedVectors

from sortedcontainers import SortedDict
from collections import defaultdict, Counter
#from functools32 import lru_cache

In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
path = 'test'
path2 = 'test2'
allFiles = glob.glob(path + "/*.tsv")
tokenizer = nltk.data.load('tokenizers/punkt/dutch.pickle')

In [7]:
def cleaning(text):
    text = re.sub("[^a-zA-Z]"," ", text)
    words = text.lower().split()
    return(words)

def article_to_sentences(text):
    #raw_sentences = tokenizer.tokenize(text.strip())
    raw_sentences = text.split('.')
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleaning(raw_sentence))
    return sentences

In [45]:
def getSentencesForYear(year):
    for file_ in allFiles:
        filename = re.sub(path + '/', '', file_)
        filename = filename[12:]
        if filename.startswith(str(year)):
            df = pd.read_csv(file_, sep='\t', encoding='utf-8', header=None)
            df.columns = ['date', 'page', 'size', 'min_x', 'min_y', 'max_x', 'max_y', 'w', 'h', 'image_url', 'ocr_url', 'ocr']
            df = df[df['ocr'].notnull()] #remove empty rows
            sentences = df['ocr'].apply(article_to_sentences)
            sentences.to_csv(path2 + '/'+ 'sentences' + filename, index=False, sep='\t', encoding='utf-8')
            sentences = [item for sublist in sentences for item in sublist]
            bigram_transformer = gensim.models.Phrases(sentences)
            bigram = gensim.models.phrases.Phraser(bigram_transformer)
            corpus = list(bigram[sentences])
    return corpus
        

def getSentencesInRange(startY, endY):
    return [s for year in range(startY, endY) for s in getSentencesForYear(year)]

In [46]:
corpus = getSentencesForYear(1975)

2018-06-21 14:30:40,559 : INFO : collecting all words and their counts
2018-06-21 14:30:40,559 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-06-21 14:30:40,695 : INFO : PROGRESS: at sentence #10000, processed 85293 words and 74559 word types
2018-06-21 14:30:40,816 : INFO : PROGRESS: at sentence #20000, processed 154910 words and 119229 word types
2018-06-21 14:30:40,949 : INFO : PROGRESS: at sentence #30000, processed 235177 words and 168018 word types
2018-06-21 14:30:41,133 : INFO : PROGRESS: at sentence #40000, processed 338613 words and 228135 word types
2018-06-21 14:30:41,269 : INFO : PROGRESS: at sentence #50000, processed 416944 words and 267250 word types
2018-06-21 14:30:41,377 : INFO : PROGRESS: at sentence #60000, processed 479848 words and 297767 word types
2018-06-21 14:30:41,536 : INFO : PROGRESS: at sentence #70000, processed 572392 words and 344392 word types
2018-06-21 14:30:41,724 : INFO : PROGRESS: at sentence #80000, processed 669224 w

2018-06-21 14:30:51,084 : INFO : PROGRESS: at sentence #740000, processed 5735680 words and 2227461 word types
2018-06-21 14:30:51,249 : INFO : PROGRESS: at sentence #750000, processed 5825324 words and 2257911 word types
2018-06-21 14:30:51,396 : INFO : PROGRESS: at sentence #760000, processed 5907253 words and 2282687 word types
2018-06-21 14:30:51,557 : INFO : PROGRESS: at sentence #770000, processed 5996091 words and 2309768 word types
2018-06-21 14:30:51,700 : INFO : PROGRESS: at sentence #780000, processed 6076005 words and 2335379 word types
2018-06-21 14:30:51,839 : INFO : PROGRESS: at sentence #790000, processed 6153569 words and 2359903 word types
2018-06-21 14:30:51,988 : INFO : PROGRESS: at sentence #800000, processed 6235951 words and 2385173 word types
2018-06-21 14:30:52,144 : INFO : PROGRESS: at sentence #810000, processed 6318015 words and 2410793 word types
2018-06-21 14:30:52,274 : INFO : PROGRESS: at sentence #820000, processed 6390171 words and 2430723 word types
2

2018-06-21 14:31:02,974 : INFO : PROGRESS: at sentence #1480000, processed 11985806 words and 4014122 word types
2018-06-21 14:31:03,109 : INFO : PROGRESS: at sentence #1490000, processed 12058658 words and 4031484 word types
2018-06-21 14:31:03,222 : INFO : PROGRESS: at sentence #1500000, processed 12119737 words and 4045078 word types
2018-06-21 14:31:03,407 : INFO : PROGRESS: at sentence #1510000, processed 12224642 words and 4074313 word types
2018-06-21 14:31:03,531 : INFO : PROGRESS: at sentence #1520000, processed 12295553 words and 4092017 word types
2018-06-21 14:31:03,653 : INFO : PROGRESS: at sentence #1530000, processed 12367100 words and 4109023 word types
2018-06-21 14:31:03,825 : INFO : PROGRESS: at sentence #1540000, processed 12464646 words and 4134761 word types
2018-06-21 14:31:03,983 : INFO : PROGRESS: at sentence #1550000, processed 12554951 words and 4158199 word types
2018-06-21 14:31:04,095 : INFO : PROGRESS: at sentence #1560000, processed 12615653 words and 41

In [ ]:
yearsInModel = 1    
stepYears = 1
modelFolder = 'tempModels_vk'

y0 = 1975
yN = 1976

for year in range(y0, yN-yearsInModel+1, stepYears):
    startY = year
    endY = year + yearsInModel
    modelName = modelFolder + '/%d_%d.w2v'%(year,year+yearsInModel)
    print('Building Model: ', modelName)
    
    corpus = getSentencesInRange(startY, endY)

    model = gensim.models.Word2Vec(min_count=10, size = 200, iter=20, alpha=0.025, window = 10, workers =7, sg=1)
    model.build_vocab(corpus)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)
    
    print('....saving')
    model.init_sims(replace=True)
    model.wv.save_word2vec_format(modelName, binary=True)

Building Model:  tempModels_vk/1975_1976.w2v


2018-06-21 15:21:05,487 : INFO : collecting all words and their counts
2018-06-21 15:21:05,490 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-06-21 15:21:05,698 : INFO : PROGRESS: at sentence #10000, processed 85293 words and 74559 word types
2018-06-21 15:21:05,849 : INFO : PROGRESS: at sentence #20000, processed 154910 words and 119229 word types
2018-06-21 15:21:06,046 : INFO : PROGRESS: at sentence #30000, processed 235177 words and 168018 word types
2018-06-21 15:21:06,318 : INFO : PROGRESS: at sentence #40000, processed 338613 words and 228135 word types
2018-06-21 15:21:06,521 : INFO : PROGRESS: at sentence #50000, processed 416944 words and 267250 word types
2018-06-21 15:21:06,675 : INFO : PROGRESS: at sentence #60000, processed 479848 words and 297767 word types
2018-06-21 15:21:06,871 : INFO : PROGRESS: at sentence #70000, processed 572392 words and 344392 word types
2018-06-21 15:21:07,122 : INFO : PROGRESS: at sentence #80000, processed 669224 w

2018-06-21 15:21:19,393 : INFO : PROGRESS: at sentence #740000, processed 5735680 words and 2227461 word types
2018-06-21 15:21:19,584 : INFO : PROGRESS: at sentence #750000, processed 5825324 words and 2257911 word types
2018-06-21 15:21:19,782 : INFO : PROGRESS: at sentence #760000, processed 5907253 words and 2282687 word types
2018-06-21 15:21:20,001 : INFO : PROGRESS: at sentence #770000, processed 5996091 words and 2309768 word types
2018-06-21 15:21:20,196 : INFO : PROGRESS: at sentence #780000, processed 6076005 words and 2335379 word types
2018-06-21 15:21:20,401 : INFO : PROGRESS: at sentence #790000, processed 6153569 words and 2359903 word types
2018-06-21 15:21:20,611 : INFO : PROGRESS: at sentence #800000, processed 6235951 words and 2385173 word types
2018-06-21 15:21:20,819 : INFO : PROGRESS: at sentence #810000, processed 6318015 words and 2410793 word types
2018-06-21 15:21:20,989 : INFO : PROGRESS: at sentence #820000, processed 6390171 words and 2430723 word types
2

2018-06-21 15:21:34,523 : INFO : PROGRESS: at sentence #1480000, processed 11985806 words and 4014122 word types
2018-06-21 15:21:34,740 : INFO : PROGRESS: at sentence #1490000, processed 12058658 words and 4031484 word types
2018-06-21 15:21:34,899 : INFO : PROGRESS: at sentence #1500000, processed 12119737 words and 4045078 word types
2018-06-21 15:21:35,214 : INFO : PROGRESS: at sentence #1510000, processed 12224642 words and 4074313 word types
2018-06-21 15:21:35,414 : INFO : PROGRESS: at sentence #1520000, processed 12295553 words and 4092017 word types
2018-06-21 15:21:35,631 : INFO : PROGRESS: at sentence #1530000, processed 12367100 words and 4109023 word types
2018-06-21 15:21:35,905 : INFO : PROGRESS: at sentence #1540000, processed 12464646 words and 4134761 word types
2018-06-21 15:21:36,197 : INFO : PROGRESS: at sentence #1550000, processed 12554951 words and 4158199 word types
2018-06-21 15:21:36,390 : INFO : PROGRESS: at sentence #1560000, processed 12615653 words and 41

2018-06-21 15:23:24,351 : INFO : PROGRESS: at sentence #400000, processed 2979206 words, keeping 228603 word types
2018-06-21 15:23:24,375 : INFO : PROGRESS: at sentence #410000, processed 3052069 words, keeping 232098 word types
2018-06-21 15:23:24,399 : INFO : PROGRESS: at sentence #420000, processed 3125791 words, keeping 235789 word types
2018-06-21 15:23:24,419 : INFO : PROGRESS: at sentence #430000, processed 3177847 words, keeping 238470 word types
2018-06-21 15:23:24,442 : INFO : PROGRESS: at sentence #440000, processed 3236682 words, keeping 241686 word types
2018-06-21 15:23:24,466 : INFO : PROGRESS: at sentence #450000, processed 3288733 words, keeping 244468 word types
2018-06-21 15:23:24,495 : INFO : PROGRESS: at sentence #460000, processed 3375157 words, keeping 248605 word types
2018-06-21 15:23:24,529 : INFO : PROGRESS: at sentence #470000, processed 3456403 words, keeping 251952 word types
2018-06-21 15:23:24,550 : INFO : PROGRESS: at sentence #480000, processed 350035

...

2018-06-21 15:24:50,944 : INFO : PROGRESS: at 11.97% examples, 277357 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:51,980 : INFO : PROGRESS: at 12.12% examples, 277448 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:52,988 : INFO : PROGRESS: at 12.28% examples, 277546 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:53,995 : INFO : PROGRESS: at 12.44% examples, 277822 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:54,997 : INFO : PROGRESS: at 12.58% examples, 277941 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:24:55,998 : INFO : PROGRESS: at 12.73% examples, 277987 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:57,013 : INFO : PROGRESS: at 12.87% examples, 277733 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:24:58,019 : INFO : PROGRESS: at 13.01% examples, 277689 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:24:59,077 : INFO : PROGRESS: at 13.15% examples, 277545 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:25:00,100 : INFO : PROGRESS: at 13.29% examples, 2

2018-06-21 15:26:11,739 : INFO : PROGRESS: at 23.09% examples, 269052 words/s, in_qsize 13, out_qsize 1
2018-06-21 15:26:12,759 : INFO : PROGRESS: at 23.24% examples, 269145 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:26:13,783 : INFO : PROGRESS: at 23.38% examples, 269174 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:26:14,806 : INFO : PROGRESS: at 23.53% examples, 269162 words/s, in_qsize 13, out_qsize 2
2018-06-21 15:26:15,824 : INFO : PROGRESS: at 23.66% examples, 269201 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:26:16,852 : INFO : PROGRESS: at 23.81% examples, 269312 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:26:17,867 : INFO : PROGRESS: at 23.93% examples, 269228 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:26:18,918 : INFO : PROGRESS: at 24.06% examples, 269086 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:26:19,919 : INFO : PROGRESS: at 24.20% examples, 269109 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:26:20,944 : INFO : PROGRESS: at 24.33% examples, 2

2018-06-21 15:27:32,408 : INFO : PROGRESS: at 34.78% examples, 271102 words/s, in_qsize 14, out_qsize 1
2018-06-21 15:27:33,436 : INFO : PROGRESS: at 34.95% examples, 271264 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:34,460 : INFO : PROGRESS: at 35.11% examples, 271456 words/s, in_qsize 14, out_qsize 1
2018-06-21 15:27:35,493 : INFO : PROGRESS: at 35.28% examples, 271579 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:36,517 : INFO : PROGRESS: at 35.43% examples, 271648 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:37,547 : INFO : PROGRESS: at 35.59% examples, 271768 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:38,548 : INFO : PROGRESS: at 35.74% examples, 271717 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:39,613 : INFO : PROGRESS: at 35.83% examples, 271331 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:40,623 : INFO : PROGRESS: at 35.93% examples, 270955 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:27:41,660 : INFO : PROGRESS: at 36.06% examples, 2

2018-06-21 15:28:53,961 : INFO : PROGRESS: at 44.95% examples, 262144 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:28:55,006 : INFO : PROGRESS: at 45.08% examples, 262078 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:28:56,013 : INFO : PROGRESS: at 45.20% examples, 262001 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:28:57,073 : INFO : PROGRESS: at 45.34% examples, 261968 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:28:58,129 : INFO : PROGRESS: at 45.48% examples, 261956 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:28:59,182 : INFO : PROGRESS: at 45.62% examples, 261926 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:29:00,226 : INFO : PROGRESS: at 45.76% examples, 261906 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:29:01,266 : INFO : PROGRESS: at 45.91% examples, 261887 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:29:02,291 : INFO : PROGRESS: at 46.05% examples, 261832 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:29:03,292 : INFO : PROGRESS: at 46.19% examples, 2

2018-06-21 15:30:15,400 : INFO : PROGRESS: at 56.64% examples, 263654 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:30:16,449 : INFO : PROGRESS: at 56.84% examples, 263826 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:30:17,478 : INFO : PROGRESS: at 57.02% examples, 263938 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:30:18,565 : INFO : PROGRESS: at 57.19% examples, 264027 words/s, in_qsize 14, out_qsize 0
2018-06-21 15:30:19,634 : INFO : PROGRESS: at 57.36% examples, 264114 words/s, in_qsize 12, out_qsize 1
2018-06-21 15:30:20,665 : INFO : PROGRESS: at 57.52% examples, 264292 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:30:21,696 : INFO : PROGRESS: at 57.70% examples, 264403 words/s, in_qsize 13, out_qsize 1
2018-06-21 15:30:22,698 : INFO : PROGRESS: at 57.86% examples, 264546 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:30:23,704 : INFO : PROGRESS: at 58.03% examples, 264691 words/s, in_qsize 13, out_qsize 0
2018-06-21 15:30:24,712 : INFO : PROGRESS: at 58.18% examples, 2

In [ ]:
model.most_similar('verenigde_staten')

In [ ]:
#model.save('/tmp/model_1975')
model_1975 = gensim.models.Word2Vec.load('/tmp/model_1975')

In [ ]:
model.most_similar('kernoorlog')

In [ ]:
#model = gensim.models.Word2Vec(min_count=10, size = 200, iter=10, alpha=0.025, window = 10, workers = 7)
model.build_vocab(sentences_1985, update=True)
model.train(sentences_1985, total_examples=model.corpus_count, epochs=model.iter)
#model.init_sims(replace=True) 

In [ ]:
#model_1975.most_similar('man')
model.most_similar('man')

In [ ]:
def _getRelatedTerms(model, seedTerms, maxRelatedTerms):
    queries = []
    threads = []

    for term in seedTerms:
        t = threading.Thread(target=_getRelatedTermsThread,
                             args=(model, term, maxRelatedTerms, queries))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    queries.sort()
    return queries

def _getRelatedTermsThread(model, term, maxRelatedTerms, queries,
                           ):
    try:
        newTerms = model.most_similar(term, topn=maxRelatedTerms)

        # list.append is thread safe, so we should be ok
        queries.append((term, newTerms))
    except KeyError:
        queries.append((term, []))
        pass

def _pruned(pairs, words):
    '''Returns a list of (word, weight) tuples which is the same as the given
    list (pairs), except containing only words in the set (words)'''
    return [(word, weight) for word, weight in pairs if word in words]


def _getCommonTerms(terms, N):
    '''Return a the top N terms of the given list. Terms are given as a
    dictionary of { term: weight } and the top terms are the terms with the
    highest weights.'''
    termCounter = Counter(terms)
    topTerms = termCounter.most_common(N)
    return topTerms


    


In [ ]:
def _trackCore(model, seedTerms, maxTerms=250, maxRelatedTerms=10,
                   minSim=0.4, wordBoost=1.0, reward=lambda x: 1.0):
        '''Given a list of seed terms, queries the given model to produce a
        list of terms. A dictionary of links is also returned as a dictionary:
        { seed: [(word,weight),...]}'''
        dRelatedTerms = defaultdict(float)
        links = defaultdict(list)

        relatedTermQueries = _getRelatedTerms(
            model, seedTerms, maxRelatedTerms)

        # Get the first tier related terms
        for term, newTerms in relatedTermQueries:
            # The terms are always related to themselves
            dRelatedTerms[term] = wordBoost
            links[term].append((term, 0.0))

            for newTerm, tSim in newTerms:
                if tSim < minSim:
                    break
                dRelatedTerms[newTerm] += reward(tSim)
                links[term].append((newTerm, tSim))

        # Select the top N terms with biggest weights (where N=maxTerms)
        topTerms = _getCommonTerms(dRelatedTerms, maxTerms)

        selectedTerms = set(word for word, weight in topTerms)
        links = {seed: _pruned(pairs, selectedTerms)
                 for seed, pairs in links.items()}
        return topTerms, links

In [ ]:
seedTerms = ['president']

In [ ]:
_trackCore(model_1985, seedTerms)

In [ ]:
women_vector = 0
for term in topTerms:
    women_vector += model_vk[term[0]]

In [ ]:
model_vk.similar_by_vector(women_vector, topn=20, restrict_vocab=None)

In [ ]:
annoy_index = AnnoyIndexer(model, 1000)

In [ ]:
model_nrc.most_similar(['politiek'], topn=15)

In [ ]:
model_vk.most_similar(['politiek'], topn=15, indexer=annoy_index)


In [ ]:
def get_vec(model, word):
    if word in model.vocab:
        vec = model[word]
    else:
        vec = None
    return vec
    

In [ ]:
#cosine similarity two words
model.wv.similarity('amerika', 'oorlog')

In [ ]:
model.wv.n_similarity(['amerika','president'],['mens','abortus'])

In [ ]:
model.wv.doesnt_match(['amerika','frisdrank','president'])

In [ ]:
#model.most_similar(positive=['vrouw'], negative=['man'], topn=50)
model.most_similar(positive=['man'], negative=['vrouw'], topn=50)

In [ ]:
model.most_similar(positive=['vrouw', 'vrouwen','man','mannen'], topn=50)

In [ ]:
her_tokens = [token for token,weight in model.wv.most_similar(positive=['vrouw'], negative=['man'], topn=50)]

In [ ]:
her_tokens

In [ ]:
# Get the vector for each sampled word
vectors = [model[word] for word in her_tokens]

In [ ]:
from sklearn.metrics import pairwise
dist_matrix = pairwise.pairwise_distances(vectors, metric='cosine')

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components = 2, dissimilarity='precomputed')
embeddings = mds.fit_transform(dist_matrix)

In [ ]:
import matplotlib.pyplot as plt
_, ax = plt.subplots(figsize=(10,10))
ax.scatter(embeddings[:,0], embeddings[:,1], alpha=0)
for i in range(len(vectors)):
    ax.annotate(her_tokens[i], ((embeddings[i,0], embeddings[i,1])))

In [ ]:

# Create functions that reject vectors, find nearest words

def vector_rejector(model, word1, word2):
    vector1 = model[word1]
    vector2 = model[word2]
    # Rejects vector_2 from vector_1
    # Note that vectors are passed in, not words; also mean of
    # several vectors could be passed as a single argument
    projection = np.dot(vector1,vector2) * vector2
    return vector1 - projection

In [ ]:
she_vector = vector_rejector(model, 'vrouw', 'man')

In [ ]:
model.similar_by_vector(she_vector, topn=20, restrict_vocab=None)